In [5]:
import pandas as pd
import yfinance as yf
import backtrader as bt
import matplotlib.pyplot as plt
import datetime

In [6]:
eurgbp = pd.read_csv('eurgbp data.csv', index_col=0, parse_dates=True)

In [7]:
eurgbp = pd.read_csv('spy 1s data from 2024.csv', index_col=0, parse_dates=True)

FileNotFoundError: [Errno 2] No such file or directory: 'spy 1s data from 2024.csv'

In [7]:
class BollingerStrategy(bt.Strategy):
    params = (
        ('period', 60),
        ('devfactor', 5),
    )

    def __init__(self):
        self.boll = bt.indicators.BollingerBands(period=self.p.period, devfactor=self.p.devfactor)

    def next(self):
        if not self.position:
            if self.data.close > self.boll.lines.top:
                self.sell()
            if self.data.close < self.boll.lines.bot:
                self.buy()

        else:
            if self.position.size > 0 and self.data.close > self.boll.lines.mid:
                self.close()
            if self.position.size < 0 and self.data.close < self.boll.lines.mid:
                self.close()
    
    def notify_order(self, order):
        if order.status == order.Completed:
            action = 'BUY' if order.isbuy() else 'SELL'
            print(f'{action} EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}')
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print('Order Canceled/Margin/Rejected')

    def notify_trade(self, trade):
        if trade.isclosed:
            print(f'Trade PNL: {trade.pnl:.2f}')


In [8]:
#Avvio Cerebro e importo i dati
cerebro = bt.Cerebro(stdstats=False)
cerebro = bt.Cerebro(cheat_on_open=True) #per calcolare la size basata sulla open dopo (all in method)


bt_data = bt.feeds.PandasData(dataname=eurgbp, timeframe = bt.TimeFrame.Minutes)
cerebro.adddata(bt_data)
cerebro.addstrategy(BollingerStrategy)
cerebro.addsizer(bt.sizers.PercentSizer, percents=90)

#Aggiungo lo slippage
cerebro.broker.set_slippage_fixed(0.000125, slip_open=True, slip_match=True, slip_out=True) #ho messo 1.25 pips come spread (media di ICMarkets.eu)

#Aggiugni gli analyzers e gli observers
cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
cerebro.addanalyzer(bt.analyzers.SQN)


results = cerebro.run()
firstStrat = results[0]

cerebro.plot(iplot=False, volume=False, width=50, height=20, style='candle')
plt.show()

BUY EXECUTED, Price: 0.88, Cost: 9001.27
SELL EXECUTED, Price: 0.88, Cost: 9001.27
Trade PNL: -3.46
SELL EXECUTED, Price: 0.88, Cost: -8995.51
BUY EXECUTED, Price: 0.88, Cost: -8995.51
Trade PNL: 9.35
BUY EXECUTED, Price: 0.88, Cost: 9006.47
SELL EXECUTED, Price: 0.88, Cost: 9006.47
Trade PNL: -4.18
BUY EXECUTED, Price: 0.88, Cost: 9011.79
SELL EXECUTED, Price: 0.88, Cost: 9011.79
Trade PNL: 6.83
BUY EXECUTED, Price: 0.88, Cost: 9007.74
SELL EXECUTED, Price: 0.88, Cost: 9007.74
Trade PNL: 0.20
BUY EXECUTED, Price: 0.88, Cost: 9009.15
SELL EXECUTED, Price: 0.88, Cost: 9009.15
Trade PNL: 0.51
BUY EXECUTED, Price: 0.89, Cost: 9006.86
SELL EXECUTED, Price: 0.89, Cost: 9006.86
Trade PNL: 9.34
BUY EXECUTED, Price: 0.89, Cost: 9016.08
SELL EXECUTED, Price: 0.89, Cost: 9016.08
Trade PNL: 2.65
BUY EXECUTED, Price: 0.88, Cost: 9020.62
SELL EXECUTED, Price: 0.88, Cost: 9020.62
Trade PNL: 12.04
SELL EXECUTED, Price: 0.88, Cost: -9029.08
BUY EXECUTED, Price: 0.88, Cost: -9029.08
Trade PNL: -11.55
S

In [9]:
for x in firstStrat.analyzers:
    x.print()

TradeAnalyzer:
  -----------------------------------------------------------------------------
  - total:
    - total: 324
    - open: 0
    - closed: 324
  -----------------------------------------------------------------------------
  - streak:
    +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    - won:
      - current: 1
      - longest: 13
    +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    - lost:
      - current: 0
      - longest: 4
  -----------------------------------------------------------------------------
  - pnl:
    +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    - gross:
      - total: 837.2963346101199
      - average: 2.5842479463275305
    +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    - net:
      - total: 837.2963346101199
      - average: 2.5842479463275305
  -----------------------------------------------------------------------------
  